# Adding context to your prompts - Retrieval Augmented Generation (RAG)

## Install OpenAI .NET SDK & other NuGet packages

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.6"
#r "nuget: Microsoft.DotNet.Interactive, 1.0.0-beta.23313.2"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.6 Microsoft.DotNet.Interactive, 1.0.0-beta.23313.2

In [2]:
using System.Collections.Generic;
using System.IO;
using System.Linq;
using System.Text.Json;
using Azure.AI.OpenAI;
using Microsoft.DotNet.Interactive;

## Get Azure OpenAI Service credentials

In [3]:
var endpoint = Environment.GetEnvironmentVariable("AOAI_ENDPOINT");
var key = Environment.GetEnvironmentVariable("AOAI_KEY");
var deploymentId = Environment.GetEnvironmentVariable("AOAI_DEPLOYMENTID");
var embeddingDeploymentId = Environment.GetEnvironmentVariable("AOAI_EMBEDDINGS_DEPLOYMENTID");

## Configure OpenAI client

In [4]:
var client = new OpenAIClient(new Uri(endpoint), new Azure.AzureKeyCredential(key));

## Define system prompt

In [5]:
var systemPrompt = 
    """
    You are an AI Assistant that helps user answer questions about their data. 
    
    You have access to articles to help you answer questions. 
    
    Use only the contents from these documents to answer questions. 
    """;

## Define Data Objects

In [6]:
class Metadata
{
    public string Title {get;set;}
    public string Text  {get;set;}
    public long WikiId  {get;set;}
    public int ParagraphId  {get;set;}
}


class Document
{
    public float[] Embedding {get;set;}
    public Metadata Metadata {get;set;}
}

## Utility Functions

In [7]:
string FormatMessage(ChatMessage m)
{
    if(m.Role == ChatRole.System)
        return $"System: {m.Content}";
    else if(m.Role == ChatRole.User)
        return $"User: {m.Content}";
    else if(m.Role == ChatRole.Assistant)
        return $"Assistant: {m.Content}";
    else
        return m.Content;
}

var RenderUserPrompt = (List<ChatMessage> history, string[] documents, string query) => 
    {
        var chatMessages = 
            history
                .Select(FormatMessage)
                .Select(x => $"{x}\n")
                .ToArray();

        var sources = String.Join('\n', documents);

        var template = 
            $"""
            ## Chat History

            {String.Join('\n',chatMessages)}

            ## Sources

            {sources}

            ## Query
            {query}
            """;

        return template;
    };

var CosineSimilarity = (float[] vectorA, float[] vectorB) => 
{
        if (vectorA == null || vectorB == null)
            throw new ArgumentNullException("Input vectors cannot be null.");

        if (vectorA.Length != vectorB.Length)
            throw new ArgumentException("Input vectors must have the same length.");

        double dotProduct = 0;
        double normA = 0;
        double normB = 0;

        for (int i = 0; i < vectorA.Length; i++)
        {
            dotProduct += vectorA[i] * vectorB[i];
            normA += vectorA[i] * vectorA[i];
            normB += vectorB[i] * vectorB[i];
        }

        if (normA == 0 || normB == 0)
            throw new ArgumentException("Input vectors cannot be zero vectors.");

        return dotProduct / (Math.Sqrt(normA) * Math.Sqrt(normB));
};

var GetTopKDocuments = (Document[] sources, float[] userQueryEmbedding, int k) => 
{
    return sources
        .Select(document => 
            {
                var similarity = CosineSimilarity(document.Embedding, userQueryEmbedding);
                return (similarity, document);
            })
        .OrderByDescending(x => x.Item1)
        .Take(k)
        .Select(x => x.Item2)
        .ToArray();
};

## Load data

In [8]:
var data = File.ReadAllText("../Data/embeddings.json");
var sources = JsonSerializer.Deserialize<Document[]>(data);

## Display first data entry

In [9]:
sources.First()

Submission#6+Document Embedding [ 0.0150072165, 0.009965043, 0.010806452, -0.031345617, -0.00073584024, -0.010448539, -0.0044425125, -0.015120241, -0.016526775, 0.001671829, 0.02639763, 0.014253716, -0.034686133, -0.01564769, -0.046415623, 0.0055319485, 0.018397968, 0.007905475, 0.007930592, -0.023685029 ... (1516 more) ] Metadata Submission#6+Metadata Title 24-hour clock Text The 24-hour clock is a way of telling the time in which the day runs from midnight to midnight and is divided into 24 hours, numbered from 0 to 23. It does not use a.m. or p.m. This system is also referred to (only in the US and the English speaking parts of Canada) as military time or (only in the United Kingdom and now very rarely) as continental time. In some parts of the world, it is called railway time. Also, the international standard notation of time (ISO 8601) is based on this format. WikiId 9985 ParagraphId 0

## Configure Chat Completion options

In [10]:
var options = new ChatCompletionsOptions
{
    MaxTokens=400,
    Temperature=1f,
    FrequencyPenalty=0.0f,
    PresencePenalty=0.0f,
    NucleusSamplingFactor = 0.95f // Top P
};

## Initialize chat history

In [11]:
List<ChatMessage> messages = new List<ChatMessage>();
var systemMessage = new ChatMessage(ChatRole.System, systemPrompt);
messages.Add(systemMessage);
options.Messages.Add(systemMessage);

## Start chat

In [12]:
var chatting = true;

In [13]:
Console.WriteLine($"{FormatMessage(systemMessage)}");

while(chatting)
{
    //Get User input
    var userInput = await Kernel.GetInputAsync("Please enter your prompt. Press 'q' to quit.");
    if(userInput.ToLowerInvariant() == "q")
    {
        chatting = false;
        break;    
    }
    
    // Query relevant documents
    var embeddingResponse = await client.GetEmbeddingsAsync(embeddingDeploymentId, new EmbeddingsOptions(userInput));
    var inputEmbedding = embeddingResponse.Value.Data[0].Embedding.ToArray();
    var docs = 
        GetTopKDocuments(sources, inputEmbedding, 3)
            .Select(doc => doc.Metadata.Text)
            .ToArray(); 

    // Generate User Prompt
    messages.Add(new ChatMessage(ChatRole.User, userInput));
    var query = RenderUserPrompt(messages, docs, userInput);
    var userMessage = new ChatMessage(ChatRole.User, query);
    options.Messages.Add(userMessage);
    Console.WriteLine(query);
    
    // Generate response
    var assistantResponse = await client.GetChatCompletionsAsync(deploymentId, options);
    var response = assistantResponse.Value.Choices[0].Message.Content;
    var assistantMessage = new ChatMessage(ChatRole.Assistant, response);
    Console.WriteLine($"\n## Response\n{FormatMessage(assistantMessage)}\n");
    messages.Add(assistantMessage);
    options.Messages.Add(assistantMessage);
    Console.WriteLine("-----");
    Console.WriteLine("");
};

messages.Clear();
options.Messages.Clear();

System: You are an AI Assistant that helps user answer questions about their data. 

You have access to articles to help you answer questions. 

Use only the contents from these documents to answer questions. 
## Chat History

System: You are an AI Assistant that helps user answer questions about their data. 

You have access to articles to help you answer questions. 

Use only the contents from these documents to answer questions. 

User: How many goals did Cristiano Ronaldo score in 2014?


## Sources

In the 2013-14 season, Ronaldo broke the record for most goals in one Champions League season by scoring his 17th goal with a penalty in extra time in the final against Atl?tico Madrid that Real Madrid won 4?1. The previous record was 14 goals, set by Messi in the 2011-12 season.
In the 2014-15 season, Ronaldo set a new personal record by scoring 61 goals in all competitions. This achievement helped him win his second Ballon d'Or. He scored five goals in one match for the first time in